In [118]:
import pandas as pd
from typing import Any
import re

## CLEANING TASKS <br>
1.	Standardize column names (snake_case) <br>
2.	Strip whitespace from all string fields <br>
3.	Normalize missing values (NaN, empty string, “-”, “NA”, “n/a”) <br>
4.	Convert Amount In USD to numeric <br>
5.	Convert Date to datetime <br>
6.	Clean city names (Bangalore/Bengaluru, Bombay/Mumbai, etc.) <br>
7.	Extract year from Date <br>
8.	Standardize Investment Type (Seed/Seed Funding → Seed) <br>
9.	Split multiple investors into lists <br>
10.	Clean Startup Name (remove suffixes like “ Pvt Ltd”, “ Limited”) <br>



In [119]:
df = pd.read_csv('/Users/olixstudios/Documents/workspace/Projects/indian-startups-transformations/data/starup.csv')
first_column_name = df.columns[0]
df = df.drop(columns=[first_column_name], errors="ignore")

In [120]:
print(df.head(52).to_dict())

{'Date': {0: '01-04-2021', 1: '05-04-2021', 2: '14-04-2021', 3: '07-04-2021', 4: '14-04-2021', 5: '04-03-2021', 6: '31-03-2021', 7: '30-03-2021', 8: '30-03-2021', 9: '23-03-2021', 10: '25-03-2021', 11: '26-03-2021', 12: '11-02-2021', 13: '22-02-2021', 14: '19-02-2021', 15: '17-02-2021', 16: '15-02-2021', 17: '12-02-2021', 18: '12-02-2021', 19: '09-02-2021', 20: '09-02-2021', 21: '15-01-2021', 22: '28-01-2021', 23: '19-01-2021', 24: '19-01-2021', 25: '18-01-2021', 26: '18-01-2021', 27: '11-01-2021', 28: '13-01-2021', 29: '15-12-2020', 30: '21-12-2020', 31: '15-12-2020', 32: '22-12-2020', 33: '15-12-2020', 34: '18-12-2020', 35: '30-11-2020', 36: '30-11-2020', 37: '29-11-2020', 38: '27-11-2020', 39: '25-11-2020', 40: '25-11-2020', 41: '25-11-2020', 42: '24-11-2020', 43: '30-11-2020', 44: '23-11-2020', 45: '28-10-2020', 46: '28-10-2020', 47: '28-10-2020', 48: '27-10-2020', 49: '27-10-2020', 50: '27-10-2020', 51: '27-10-2020'}, 'Startup': {0: "BYJU'S", 1: 'Meesho', 2: 'Swiggy', 3: 'Groww', 

In [121]:
def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    def _snake_case(s: Any) -> str:
        if not isinstance(s, str):
            s = str(s)
        s  = s.strip()
        s = s.replace(" ", "_")
        s = re.sub(r"[^\w\s]", "", s)  
        return s.lower()
    df =  df.copy()
    df.columns = [_snake_case(col) for col in df.columns]
    return df

In [122]:
def normalize_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    MISSING_TOKENS = ["N/A", "n/a", "NA", "na", "-", "--", " ", ""]
    df = df.copy()
    df.replace(to_replace=MISSING_TOKENS, value=pd.NA, inplace=True)
    return df


In [123]:
def clean_amount_column(df: pd.DataFrame, col: str = "amount") -> pd.DataFrame:
    df = df.copy()
    df[col] = df[col].astype(str).fillna("")
    #df[col] = df[col].str.replace(",", "")
    df[col] = df[col].str.strip()
    #df[col] = df[col].str.replace(r"[\$,€£¥]", "", regex=True)
    df[col] = df[col].str.replace(r"[^0-9\.\-]", '', regex=True).replace('', pd.NA)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].replace(
     to_replace=["Undisclosed", "undisclosed"],
     value=pd.NA,
    )
    return df


In [124]:
def parse_dates(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[col] = pd.to_datetime(df[col], dayfirst=True ,errors='coerce') 
    return df

In [125]:
def date_analysis(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df["year"] = df[date_col].dt.year
    df["date_missing"] = df[date_col].isna()
    return df

In [126]:
print('orig cols:', df.columns.tolist())
df = clean_column_names(df)
print('clean cols:', df.columns.tolist())
df = normalize_missing_values(df)
print('null counts:', df.isna().sum().to_dict())
df = clean_amount_column(df, col='amount')   # or amount_in_usd
print('amount dtype:', df['amount'].dtype)
df = parse_dates(df, col='date')
print('date dtype:', df['date'].dtype)
print(df[['amount','date']].head().to_dict())

orig cols: ['Date', 'Startup', 'Industry', 'Location', 'Investor', 'Type', 'Amount']
clean cols: ['date', 'startup', 'industry', 'location', 'investor', 'type', 'amount']
null counts: {'date': 5, 'startup': 5, 'industry': 5, 'location': 5, 'investor': 35, 'type': 12, 'amount': 57}
amount dtype: float64
date dtype: datetime64[ns]
{'amount': {0: 460000000.0, 1: 300000000.0, 2: 343000000.0, 3: 83000000.0, 4: 7400000.0}, 'date': {0: Timestamp('2021-04-01 00:00:00'), 1: Timestamp('2021-04-05 00:00:00'), 2: Timestamp('2021-04-14 00:00:00'), 3: Timestamp('2021-04-07 00:00:00'), 4: Timestamp('2021-04-14 00:00:00')}}


In [127]:
import numpy as np
temp = df['type'].unique()
temp = pd.DataFrame(temp)
temp



,0
0,Series F
1,Series E
2,Series J
3,Series D
4,Venture
...,...
64,Mezzanine
65,Series B (Extension)
66,Equity Based Funding
67,Private Funding


In [128]:
temp = df['type'].groupby(df['type']).size()
temp

type
 Venture - Series Unknown    1
Angel                        7
Angel Round                  1
Bridge Funding               1
Bridge Round                 1
                            ..
Venture-Series Unknown       2
pre-Seed                     1
pre-Series A                 5
pre-Series B                 1
pre-series A                 1
Name: type, Length: 68, dtype: int64

In [129]:
temp = df['type']
temp

0                 Series F
1                 Series E
2                 Series J
3                 Series D
4                  Venture
              ...         
611         Private Equity
612         Private Equity
613         Private Equity
614         Private Equity
615    Seed/ Angel Funding
Name: type, Length: 616, dtype: object

In [130]:
CANONICAL_PATTERNS = {
    r"^angel.*": "angel",
    r".*angel funding.*": "angel",
    r"^seed.*": "seed",
    r".*seed funding.*": "seed",
    r".*seed / angel.*": "seed",
    r"^pre[\s\-]?seed.*": "pre_seed",
    r"^pre[\s\-]?series a.*": "pre_seed",   # or "pre_series_a"
    r"^series [a-z0-9].*": "series",
    r".*venture.*": "venture",
    r".*private equity.*": "private_equity",
    r".*debt.*": "debt",
    r".*equity.*": "equity",
    r".*mezzanine.*": "mezzanine",
    r".*m&a.*": "ma",
    r".*bridge.*": "bridge",
    r".*secondary market.*": "secondary_market",
    r".*in progress.*": "in_progress",
    r".*unspecified.*": "unspecified",
}

def canonical_investment_type(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.lower().str.strip()

    out = s.replace(CANONICAL_PATTERNS, regex=True)
    out = out.where(out.isin(set(CANONICAL_PATTERNS.values())), other="other")

    return out


In [131]:
df["type"].value_counts()

type
Private Equity         181
Seed/ Angel Funding    119
Series A                45
Series B                31
Series C                28
                      ... 
Venture Series           1
Funding                  1
Pre-series A             1
Series G                 1
Pre Seed                 1
Name: count, Length: 68, dtype: int64

In [132]:

df["type_canonical"] = canonical_investment_type(df["type"])
df["type_canonical"].value_counts()

type_canonical
equity              188
series              147
angel               135
seed                 44
other                26
debt                 24
pre_seed             22
venture              20
in_progress           3
secondary_market      2
bridge                2
unspecified           1
ma                    1
mezzanine             1
Name: count, dtype: int64

In [133]:
SPLIT_PATTERN = r'(,|;|/|\sand\s|\sAND\s|\n)'
def split_investors(df: pd.DataFrame, col: str = "investor") -> pd.DataFrame:
    out = df.copy()
    # ensure we handle missing values BEFORE casting to str
    series = out[col].where(out[col].notna(), "")
    series = series.astype(str).str.strip()
    split_pattern = r",\s*|;\s*|/\s*|\s+and\s+|\s+AND\s+|\n+"
    out["investor_list"] = series.str.split(split_pattern)
    # clean each list: strip, drop empty, drop 'others'
    def clean_list(lst):
        return [x.strip() for x in lst if x and x.strip() and x.strip().lower() != "others"]
    out["investor_list"] = out["investor_list"].apply(clean_list)
    out["investor_count"] = out["investor_list"].apply(len)
    return out

df = split_investors(df, col="investor")
df[["investor", "investor_list", "investor_count"]].head().to_dict()

{'investor': {0: 'Innoven Capital',
  1: 'SoftBank Vision Fund 2',
  2: 'Amansa Holdings, Carmignac, Falcon Edge Capital, Goldman Sachs, Think Investments',
  3: 'MC Global Edtech, B Capital, Baron, others',
  4: 'Hindustan Media Ventures'},
 'investor_list': {0: ['Innoven Capital'],
  1: ['SoftBank Vision Fund 2'],
  2: ['Amansa Holdings',
   'Carmignac',
   'Falcon Edge Capital',
   'Goldman Sachs',
   'Think Investments'],
  3: ['MC Global Edtech', 'B Capital', 'Baron'],
  4: ['Hindustan Media Ventures']},
 'investor_count': {0: 1, 1: 1, 2: 5, 3: 3, 4: 1}}

In [134]:
temp = df['startup'].nunique() == len(df['startup'])
print(df['startup'].nunique())
print(len(df['startup']))
temp

326
616


False

In [135]:
gp = df['startup'].groupby(df['startup']).size().sort_values(ascending=False)
pd.DataFrame(gp)
gp

startup
Nykaa                6
Shuttl               6
LetsTransport        5
Udaan                5
BYJU'S               5
                    ..
 InCred Finance      1
LetsTransport\n\n    1
Licious              1
Lo! Foods            1
 Guiddoo             1
Name: startup, Length: 326, dtype: int64

In [136]:
SUFFIX_PATTERN = r"""
    \s+(
        pvt\.?\s*ltd      |   # Pvt Ltd / Pvt. Ltd
        private\s*limited |   # Private Limited
        ltd\.?            |   # Ltd / Ltd.
        limited           |   # Limited
        inc\.?            |   # Inc / Inc.
        incorporated      |
        corp\.?           |
        corporation
    )\s*$
"""    


def clean_startup_name(name: str) -> str:
    if not isinstance(name, str):
        return name

    # Normalize spaces and case
    s = name.strip()
    s = re.sub(SUFFIX_PATTERN, "", s, flags=re.IGNORECASE | re.VERBOSE)
    s = re.sub(r"\n", "", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()
df["startup_clean"] = df["startup"].apply(clean_startup_name)
df

,date,startup,industry,location,investor,type,amount,type_canonical,investor_list,investor_count,startup_clean
0,2021-04-01,BYJU'S,Edu-tech,Bengaluru,Innoven Capital,Series F,460000000.0,series,[Innoven Capital],1,BYJU'S
1,2021-04-05,Meesho,E-commerce,Bengaluru,SoftBank Vision Fund 2,Series E,300000000.0,series,[SoftBank Vision Fund 2],1,Meesho
2,2021-04-14,Swiggy,Online Food Delivery,Bengaluru,"Amansa Holdings, Carmignac, Falcon Edge Capita...",Series J,343000000.0,series,"[Amansa Holdings, Carmignac, Falcon Edge Capit...",5,Swiggy
3,2021-04-07,Groww,FinTech,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series D,83000000.0,series,"[MC Global Edtech, B Capital, Baron]",3,Groww
4,2021-04-14,Beldara,E-commerce,Mumbai,Hindustan Media Ventures,Venture,7400000.0,venture,[Hindustan Media Ventures],1,Beldara
...,...,...,...,...,...,...,...,...,...,...,...
611,2018-07-26,TheCapitalNet,Fin-Tech,Hyderabad,Lindwall Family Investments LLC (LFI),Private Equity,500000.0,equity,[Lindwall Family Investments LLC (LFI)],1,TheCapitalNet
612,2018-07-30,Shuttl,Consumer Internet,Gurugram,Amazon Alexa Fund & Dentsu Ventures,Private Equity,11000000.0,equity,[Amazon Alexa Fund & Dentsu Ventures],1,Shuttl
613,2018-07-30,Cure Fit,Consumer Internet,Bengaluru,"IDG Ventures, Accel Partners, Kalaari Capital ...",Private Equity,120000000.0,equity,"[IDG Ventures, Accel Partners, Kalaari Capital]",3,Cure Fit
614,2018-07-31,Five Star Group,Fin-Tech,Chennai,"TPG, Norwest Venture Partners, Sequoia Capital...",Private Equity,100000000.0,equity,"[TPG, Norwest Venture Partners, Sequoia Capita...",3,Five Star Group


In [137]:
def normalize_industry(s: pd.Series ) -> pd.Series:
    return s

In [138]:
INDUSTRY_PATTERNS_ORDERED = [
    (r"\bed[\s\-]?tech\b", "edtech"),
    (r"\b(edutech|edutech|edu tech)\b", "edtech"),  # extra safety
    (r"\bfin[\s\-]?tech\b|\bfinancial\b|\bfinance\b", "fintech"),
    (r"\bb2b[\s\-]?e[\s\-]?commerce\b|\be[\s\-]?commerce\b", "ecommerce"),
    # SaaS/Tech must be specific: match 'saas' or 'technology provider' or 'tech provider' NOT bare 'tech'
    (r"\bsaas\b|\btechnology\s*provider\b|\btech\s*provider\b", "saas_tech"),
    (r"\bfood\b|\bbeverage\b", "food_beverage"),
    (r"\bagri[\s\-]?tech\b|\bagritech\b", "agritech"),
    (r"\bhealth\b", "healthtech"),
    (r"\btransport(ation)?\b", "transport"),
    (r"\blifestyle\b", "lifestyle"),
    (r"\bhospital\b|\bhospitality\b", "hospitality"),
    (r"\badvertis", "advertising"),
    (r"\bdigital\b", "digital"),
    (r"\bdairy\b", "dairytech"),
    (r"\bsupply\s*chain\b", "supply_chain"),
]

In [139]:
df['industry'].unique().tolist()

['Edu-tech',
 'E-commerce',
 'Online Food Delivery',
 'FinTech',
 'Technology',
 'Edu Tech',
 'Hospitality',
 'Finance',
 'E-Commerce',
 'EdTech',
 'Financial Services',
 'Consumer Goods Company',
 'AgriTech Startup',
 'SaaS',
 'Health Tech Startup',
 'Food Startup',
 'B2B E-commerce',
 'Food and Beverage',
 'Technology Provider',
 'Supply chain ',
 'EduTech',
 'Lifestyle',
 'Transportation',
 'Edutech',
 'Fintech',
 'Dairy Tech',
 'Digital Solutions',
 'AgriTech',
 'Advertising',
 'Information Technology',
 'Personal Care',
 'Online Gaming',
 'Video Customer Experience(CX) Platform',
 'HealthTech',
 'Human Resources',
 'Automobile',
 'FoodTech',
 'Hyper-local Logistics',
 'Water Purification',
 'Airfare Marketplace',
 'Gaming and Entertainment',
 'Marketplace',
 'Automotive Rental',
 'OTT Player',
 'Industrial Robotics',
 'Rural FinTech',
 'BioTech',
 'Logistics Solution Provider',
 'Deep Tech AI',
 'Block Chain, Cryptocurrency',
 'Professional Network for Women',
 'Utility Solutions 

In [140]:
def normalize_industry(series: pd.Series) -> pd.Series:
    s = series.fillna("").astype(str).str.lower().str.strip()
    s = s.str.replace(r"[\-_/]", " ", regex=True)         # normalize separators
    s = s.str.replace(r"\s+", " ", regex=True)            # collapse spaces

    def map_one(val: str) -> str:
        for pat, canon in INDUSTRY_PATTERNS_ORDERED:
            if re.search(pat, val):
                return canon
        return "other"

    return s.apply(map_one)

In [144]:
s = normalize_industry(df['industry'])
s.unique()

array(['edtech', 'ecommerce', 'food_beverage', 'fintech', 'other',
       'hospitality', 'agritech', 'saas_tech', 'healthtech',
       'supply_chain', 'lifestyle', 'transport', 'dairytech', 'digital',
       'advertising'], dtype=object)

In [146]:
df['location'].dropna().unique()

array(['Bengaluru', 'Mumbai', 'Jaipur, Rajasthan', 'Palo Alto', 'Gurgaon',
       'London', 'New Delhi', 'Patna', 'Pune', 'Noida', 'Washington',
       'Gurugram', 'Bangalore', 'Wilmington, Delaware, United States',
       'Newark, Delaware, United States of Amercia',
       'Stanford, California, ', 'Singapore and Bangalore',
       'New York and Delhi', 'Chennai', 'Ahmedabad', 'Bhopal',
       'Hyderabad', 'San Francisco/ Bangalore', 'Faridabad',
       'San Francisco', 'San Jose,', 'Amritsar', 'Delhi', 'Kormangala',
       'Tulangan', 'Burnsville', 'Menlo Park', 'Palo Alto ',
       'Santa Monica', 'Singapore', 'Taramani', 'Andheri', 'Chembur',
       'Nairobi', 'Bengaluru ', 'Haryana', 'New York', 'Karnataka',
       'Mumbai/Bengaluru', 'Bengaluru and Gurugram', 'India/Singapore',
       'Jaipur', 'India/US', 'Nagpur', 'Indore', 'New York, Bengaluru',
       'California', 'India', 'Ahemadabad', 'Rourkela'], dtype=object)